# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 01.10.2023

Мягкий дедлайн: 23:59MSK 15.10.2023

Жесткий дедлайн: 23:59MSK 20.10.2023

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

Итогова оценка считается как
$$
min(task_1, task_2)*0.8 + max(task_1, task_2)*0.2
$$

где task_1 и task_2 - оценки за первое и второе заданиее соответсвенно.
Также, за домашнее задание выставляется 0, если не сделано нулевое или третье задание.
### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import random
device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [2]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2023-10-12 08:00:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data.txt.zip’

data.txt.zip            [         <=>        ] 201.24M  20.7MB/s    in 11s     

2023-10-12 08:00:12 (18.6 MB/s) - ‘data.txt.zip’ saved [211011981]



In [3]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


Мы вывели кусок данных, чтобы понять, насколько они пригодны для работы без изменений. Здесь ясно, что сомнительно дальше с такими данными работать, потому что как минимум есть отрицательные значения, которые не отмасштабированы, кроме того еще сразу бросается в глаза совсем разная размерность, где-то видим реально большие числа, а где-то 0.075. Ясно, что будем скейлить.

In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

Для выполнения данного задания (и всех последующих) предобработайте данные.

1. Зафиксируйте random_seed везде где только возможно. Вам предоставлена функция для этого, однако вы можете дополнить ее своими дополнениями
2. Обучите StandertScaler и предобработайте ваши данные. В следующих заданиях можете использовать другой scaler или вообще отказаться от него


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()

In [6]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [7]:
set_random_seed(42)

In [8]:
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [9]:
# подбираем гиперпарамтер альфа для Ridge-регрессии
reg_coef = []
for alpha in [0, 0.001, 0.01, 0.1, 1, 10, 20, 50]:
    model = Ridge(alpha).fit(X_train_norm, y_train)
    rmse_for_model = mean_squared_error(y_test, model.predict(X_test_norm), squared=False)
    reg_coef.append((alpha, rmse_for_model))

best_alpha, best_rmse_for_model = min(reg_coef, key=lambda x: x[1])
print(f"Best Ridge alpha = {best_alpha}\nBest Ridge RMSE = {best_rmse_for_model}")

Best Ridge alpha = 0
Best Ridge RMSE = 9.510160707488621


Лучшая константа для RMSE это среднее, посчитаем значение метрики при нем

In [10]:
from sklearn.dummy import DummyRegressor

const_model = DummyRegressor(strategy="mean")
const_model.fit(X_train_norm, y_train)
best_rmse_metric = mean_squared_error(y_test, const_model.predict(X_test_norm), squared=False)

print(f"Const model RMSE = {best_rmse_metric}")

Const model RMSE = 10.85246390513634


## Задание 1. (максимум 10 баллов)

Закрепите свои знания о том, как pytorch работает с обратным распространением ошибки, проделав следующие шаги:

1. Создайте модель линейной регрессии, которая будет состоять только из одного Linear слоя.
2. Напишите цикл обучения вашей линейной регрессии. В нем реализуйте подсчет функции потерь, сделайте шаг градиентного спуска. Запрещено использовать готовые оптимизаторы и loss-функции из библиотеки pytorch. Для подсчета градиента воспользуйтесь методом backward.
3. Запустите обучение на 10 эпохах, после каждой проверяйте значение целевой метрики на тестовой выборке.
4. Выведите на экран графики метрики и значения функции потерь на тестовой и обучающей выборке.

В данном задании нет цели побить какой-то порог по метрике. Ваша задача - убедиться в том, что ваш рукописный цикл обучения работает. Для ускорения вычислений и обучения модели можете брать только срез данных, а не весь датасет.

In [11]:
!pip install wandb
!wandb login

import wandb

wandb: Currently logged in as: grigvasilisa (grigorevaaa). Use `wandb login --relogin` to force relogin


In [12]:
def MSE(true, predicted):
    return ((true - predicted) ** 2).sum() / true.numel()

In [13]:
def RMSE(true, predicted):
    return torch.sqrt(MSE(true, predicted))

In [14]:
class LinRegDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return [
            torch.tensor(self.x[idx, :], dtype=torch.float),
            self.y[idx]
        ]

In [15]:
train_ds = LinRegDataset(X_train_norm, y_train)
test_ds = LinRegDataset(X_test_norm, y_test)

train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=50)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=50)

In [44]:
lin_model = nn.Linear(90, 1)
lin_model = lin_model.to(device)

In [43]:
# инициализируем проект
wandb.init(project="pytorch-iad_hw1")
# сохраняем параметры сетки в wandb + просим следить за градиентами сетки
wandb.watch(lin_model)

mean test loss,██▁█
mean train loss,██▁█
mean test loss,9490.59961
mean train loss,11889.45801


[]

In [45]:
set_random_seed(42)
lr = 0.01
print(f"Learning rate = {lr}")

for epoch in range(10):
    train_loss = []
    for x_train, y_train in tqdm(train_dataloader):  # берем батч из трейн лоадера
        # я обучаю модель на RMSE метрике
        y_pred = lin_model(x_train.to(device))  # делаем предсказания
        loss = RMSE(y_pred, y_train.to(device))  # считаем лосс
        train_loss.append(loss.detach().cpu().numpy() ** 2) # добавляю MSE в массив, чтобы потом посчитать точное значение RMSE
                                                            # не усреднять корни, а взять корень среднего

        loss.backward()  # считаем градиенты обратным проходом

        # вычитаем производные из параметров
        # записывать историю вычислений уже не нужно (no_grad)
        with torch.no_grad():
            lin_model.weight -= lin_model.weight.grad * lr
            lin_model.bias -= lin_model.bias.grad * lr
            # обнуляем производные
            lin_model.weight.grad.zero_()
            lin_model.bias.grad.zero_()


    test_loss = []  # сюда будем складывать средний по бачу лосс (MSE), а потом его переводить в RMSE
    with torch.no_grad():  # на валидации запрещаем фреймворку считать градиенты по параметрам
        for x_test, y_test in tqdm(test_dataloader):  # берем батч из валидационного лоадера
            y_pred = lin_model(x_test.to(device))  # делаем предсказания
            loss = MSE(y_pred, y_test.to(device))  # считаем лосс

            test_loss.append(loss.cpu().numpy())  # добавляем в массив

    # скидываем метрики на wandb
    wandb.log(
        {
            "mean train loss": np.sqrt(np.mean(train_loss)),
            "mean test loss": np.sqrt(np.mean(test_loss))
        }
    )

    # печатаем метрики
    print(
        f"Epoch: {epoch}, loss: {np.sqrt(np.mean(test_loss))}"
    )

Learning rate = 0.01


  0%|          | 0/9275 [00:00<?, ?it/s]

  0%|          | 0/1033 [00:00<?, ?it/s]

Epoch: 0, loss: 9491.22265625


  0%|          | 0/9275 [00:00<?, ?it/s]

...

  0%|          | 0/1033 [00:00<?, ?it/s]

Epoch: 8, loss: 76.10035705566406


  0%|          | 0/9275 [00:00<?, ?it/s]

  0%|          | 0/1033 [00:00<?, ?it/s]

Epoch: 9, loss: 76.10035705566406


Ссылки на графики:

https://api.wandb.ai/links/grigorevaaa/o17de5pw - test loss

https://api.wandb.ai/links/grigorevaaa/sdb0k5vo - train loss

## Задание 2. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из предыдущего задания.



Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться сигнатурами функций, которые приведены ниже. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [46]:
# заново готовим данные, чтобы избежать ошибок

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

target_scaler = StandardScaler()
y_train_norm = target_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_norm = target_scaler.transform(y_test.reshape(-1, 1))

In [47]:
from torch.optim.lr_scheduler import ExponentialLR

In [48]:
set_random_seed(42)

train_set = LinRegDataset(X_train_norm, y_train_norm)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=25)

test_set = LinRegDataset(X_test_norm, y_test_norm)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=25)

model = nn.Sequential(
    nn.Linear(
        90, 70
    ),  # линейный слой, преобразующий вектор размера 90 в вектор размера 45
    nn.ReLU(),  # нелинейность
    nn.Linear(
        70, 40
    ),
    nn.ReLU(),  # нелинейность
    nn.Linear(
        40, 20
    ),
    nn.BatchNorm1d(20), # батчнорм посередине для повышения качества
    nn.ReLU(),  # нелинейность
    nn.Linear(
        20, 10
    ),
    nn.ReLU(),  # нелинейность
    nn.Linear(
        10, 1
    ),
)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9)
criterion = MSE

In [100]:
# инициализируем проект
wandb.init(project="pytorch-iad_hw1")
# сохраняем параметры сетки в wandb + просим следить за градиентами сетки
wandb.watch(model)

[]

In [49]:
def train(model, optimizer, criterion, scheduler, train_loader, test_loader):
    """
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function
        scheduler - scheduler for optimizer step
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    """
    set_random_seed(42)

    for epoch in range(13):
        train_loss = []
        for x_train, y_train in tqdm(train_loader):
            # я обучаю модель на MSE метрике
            y_pred = model(x_train.to(device))
            loss = criterion(y_pred, y_train.to(device))
            train_loss.append(loss.detach().cpu().numpy())
            loss.backward()

            optimizer.step()  # обновляем параметры сети
            optimizer.zero_grad()  # обнуляем посчитанные градиенты параметров

        scheduler.step()


        test_loss = 0  # сюда будем складывать средний по бачу лосс
        with torch.no_grad():  # на валидации запрещаем фреймворку считать градиенты по параметрам
            y_pred = test(model, criterion, test_loader)
            test_loss = mean_squared_error(target_scaler.inverse_transform(y_pred.cpu().numpy()),
                                           y_test.reshape(-1, 1), squared=False)

        # скидываем метрики на wandb
        wandb.log(
            {
                "mean train loss (MSE)": np.mean(train_loss),
                "mean test loss (RMSE)": test_loss
            }
        )

        # печатаем метрики
        print(
            f"Epoch: {epoch}, loss: {np.mean(test_loss)}"
        )


def test(model, criterion, test_loader):
    """
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    """
    predicts = torch.tensor([])
    with torch.no_grad():
        for x_test, y_test in tqdm(test_loader):  # берем батч из валидационного лоадера
            y_pred = model(x_test.to(device))  # делаем предсказания
            predicts = torch.cat((predicts.to(device), y_pred.to(device)))

    return predicts


In [102]:
assert test(model, RMSE, test_loader).shape[0] == y_test.shape[0]

  0%|          | 0/2066 [00:00<?, ?it/s]

In [103]:
set_random_seed(42)

train(model, optimizer, criterion, scheduler, train_loader, test_loader)

  0%|          | 0/18549 [00:00<?, ?it/s]

  0%|          | 0/2066 [00:00<?, ?it/s]

Epoch: 0, loss: 8.85775638329918


  0%|          | 0/18549 [00:00<?, ?it/s]

  0%|          | 0/2066 [00:00<?, ?it/s]

...

  0%|          | 0/2066 [00:00<?, ?it/s]

Epoch: 11, loss: 8.479870939513702


  0%|          | 0/18549 [00:00<?, ?it/s]

  0%|          | 0/2066 [00:00<?, ?it/s]

Epoch: 12, loss: 8.483582754891001


In [104]:
pred = test(model, RMSE, test_loader)

  0%|          | 0/2066 [00:00<?, ?it/s]

In [105]:
pred_act = target_scaler.inverse_transform(pred.cpu().numpy())

In [108]:
print(f"RMSE for test data = {mean_squared_error(pred_act, y_test.reshape(-1, 1), squared=False)}")

RMSE for test data = 8.483582754891001


Train loss: https://api.wandb.ai/links/grigorevaaa/jvw8j70a

Test loss: https://api.wandb.ai/links/grigorevaaa/0kucxat5

## Задание 3. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

*Такого качества мне удалось добиться сквозь боль и слезы.... ладно, только боль!*

Пробовала разные архитектуры, начиная с простой двуслойной линейной и заканчивая трехслойной линейной с батчнормом и дропаутом (кстати они роняли качество) + сначала использовала для обучения ненормированный таргет. В итоге нормировка таргета очень помогла в улучшении качества (в выводе отдельно об этом напишу)

Посмотрим метрики некоторых экспов:
(я бы показала больше метрик, но за 3 дня провела столько экспов, что от сохранения их метрик тошнило)

## Эксп 1
Архитектура, с которой начинала

```
model = nn.Sequential(
     nn.Linear(
         90, 45
     ),
     nn.ReLU(),
     nn.Linear(
         45, 1
     )
 )

train_set = LinRegDataset(X_train_norm, y_train)
train_loader = DataLoader(train_set, batch_size=128)

test_set = LinRegDataset(X_test_norm, y_test)
test_loader = DataLoader(test_set, batch_size=128)

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = RMSE
```
В ней не было ничего удивительного, просто накинула еще один линейный слой в сочетании с функцией активации и долго мучилась в подборе батчсайза и шага градиентного спуска.

При батчсайзе > 25 метрика модели на тесте почти не падала или не падала вообще (это убивало мое моральное благополучие), не удавалось изменить качество даже подбором различных шагов SGD. В итоге пришла к тому что стоит оставить **batch_size = 25, а learning_rate = 0.001**. В обучении кстати использовался ненормированный таргет и обучала модель на RMSE, отсюда низкие показатели метрики по 10-20 эпохам. Примерно было так:

RMSE progress: 2048 -> 534 -> 430 -> 132 -> 112 -> ... -> 114


## Эксп 2
Вдобавок к этой архитектуре решила **пронормировать таргет** - гениальная идея, и **обучать на MSE**, а считать тестовую метрику на RMSE

Эти решения значительно повысили качество модели и она, наконец, стала выдавать реальные метрики: RMSE = +-10

## Эксп 3
Даже с значением метрики в районе 10 я все равно получала 0 баллов за задание - надо было решать проблему.

Я продолжила **экспериментировать с архитектурами, усложняя и упрощая, добавляя метод моментов в SGD, регулируя шаг шедулером, накидывая батчнормы и дропауты**. Для корректного расположения дропаут- и батчнорм- слоев прочитала следующую заметку (https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout)

Стоит отметить, что добавление метода моментов + шедулера уронили метрику до 8.82

После я добавила еще один линейный слой к архитектуре и накинула сочетание батчнорма с дропаутом - качество ухудшилось до 10.32 на 10 эпохах.
Я долго мучалась, избавилась от батчнорма и дропаута, сделала еще какие-то (незанчительные) модификации, поменяла число эпох, learning_rate и добилась метрики в 8.81

Обучение на этом этапе длилось по 30 минут и, честно, я устала. Захотелось в последней попытке закинуть в сеть вида


```
model = nn.Sequential(
    nn.Linear(
        90, 70
    ),
    nn.ReLU(),
    nn.Linear(
        70, 40
    ),
    nn.ReLU(),
    nn.Linear(
        40, 20
    ),
    nn.ReLU(),
    nn.Linear(
        20, 10
    ),
    nn.ReLU(),
    nn.Linear(
        10, 1
    ),
)
```

батчнорм слой куда-нибудь в середину и, о чудо, метрика упала до 8.75 на обучении, я изменила количество эпох в обучении и пришла к результату

**RMSE for test data = 8.483582754891001**


## Выводы
- Нормировка матрицы объектов - само собой прекрасная идея, но нормировка таргета - придумана Богом. В целом, очень логично, что модель легче и точнее обучается на нормированном таргете, тк не происходит переполнение при подсчете метрики ошибки. Но до этого еще дойти надо

- Если меняешь батчсайз, то обязательно нужно менять learning_rate и еще возможно перестраивать слегка архитектуру модели. Чем больше батч - тем быстрее проходит одна эпоха обучения, но (мой случай) большие батчи могут полностью тормозить обучение модели => батчи берем не слишком большими

- шедулер, метод моментов - одним словом, регулировка градиентного спуска очень помогает в обучении модели

- батчнорм помогает лучше нормировать признаки внутри батча, хорошо ставить его после линейного слоя

- дропаут препятствтует переобучению, но в больших количествах и при слабенькой модели сильно роняет качество. Лучше ставить его после функции активации


Кстати в статье по структуру сетей с батчнормом и дропаутом предложено две схемы:

Scheme 1

> -> CONV/FC -> ReLu(or other activation) -> Dropout -> BatchNorm -> CONV/FC ->

Scheme 2

> -> CONV/FC -> BatchNorm -> ReLu(or other activation) -> Dropout -> CONV/FC ->

На практике оказалось, что качество от Scheme 1 лучше, чем от Scheme 2


Спасибо за уделенное время!!!!